### Project Configurations
```
# Runtime params
#===================================
train: True # train new or existing model for each channel
predict: True # generate new predicts or, if False, use predictions stored locally
use_id: "2018-05-19_15.00.10"

# number of values to evaluate in each batch
batch_size: 70

# number of trailing batches to use in error calculation
window_size: 30

# Columns headers for output file
header: ["run_id", "chan_id", "spacecraft", "num_anoms", "anomaly_sequences", "class", "true_positives", 
        "false_positives", "false_negatives", "tp_sequences", "fp_sequences", "gaussian_p-value", "num_values",
        "normalized_error", "eval_time", "scores"]

# determines window size used in EWMA smoothing (percentage of total values for channel)
smoothing_perc: 0.05

# number of values surrounding an error that are brought into the sequence (promotes grouping on nearby sequences
error_buffer: 100

# LSTM parameters
# ==================================
loss_metric: 'mse'
optimizer: 'adam'
validation_split: 0.2
dropout: 0.3
lstm_batch_size: 64

# maximum number of epochs allowed (if early stopping criteria not met)
epochs: 35

# network architecture [<neurons in hidden layer>, <neurons in hidden layer>]
# Size of input layer not listed - dependent on evr modules and types included (see 'evr_modules' and 'erv_types' above)
layers: [80,80]

# Number of consequetive training iterations to allow without decreasing the val_loss by at least min_delta 
patience: 10
min_delta: 0.0003

# num previous timesteps provided to model to predict future values
l_s: 250

# number of steps ahead to predict
n_predictions: 10

# Error thresholding parameters
# ==================================

# minimum percent decrease between max errors in anomalous sequences (used for pruning)
p: 0.13
```

### Step by Step:

1. load data one by one according to the row in the labeled_anomalies.csv
2. for each set of data:

* Load Data
For example:

##### l_s = 250 (num previous timesteps provided to model to predict future values), 
##### n_predictions = 10 (number of steps ahead to predict）
data = [] 
for i in range(len(arr) - config.l_s - config.n_predictions):
    data.append(arr[i:i + config.l_s + config.n_predictions])
    
Thus:    
train_shape: (2872, 25), X_train_shape: (2612, 250, 25), y_train_shape: (2612, 10)
X_test_shape: (8245, 250, 25), y_test_shape: (8245, 10)

* Train Model
```
def get_model(anom, X_train, y_train, logger, train=False):
    '''Train LSTM model according to specifications in config.yaml or load pre-trained model.

    Args:
        anom (dict): contains all anomaly information for a given input stream
        X_train (np array): numpy array of training inputs with dimensions [timesteps, l_s, input dimensions)
        y_train (np array): numpy array of training outputs corresponding to true values following each sequence
        logger (obj): logging object
        train (bool): If False, will attempt to load existing model from repo

    Returns:
        model (obj): Trained Keras LSTM model 
    '''

    if not train and os.path.exists(os.path.join("data", config.use_id, "models", anom["chan_id"] + ".h5")):
        logger.info("Loading pre-trained model")
        return load_model(os.path.join("data", config.use_id, "models", anom["chan_id"] + ".h5"))

    elif (not train and not os.path.exists(os.path.join("data", config.use_id, "models", anom["chan_id"] + ".h5"))) or train:
        
        if not train:
            logger.info("Training new model from scratch.")

        cbs = [History(), EarlyStopping(monitor='val_loss', patience=config.patience, 
            min_delta=config.min_delta, verbose=0)]
        
        model = Sequential()

        model.add(LSTM(
            config.layers[0],
            input_shape=(None, X_train.shape[2]),
            return_sequences=True))
        model.add(Dropout(config.dropout))

        model.add(LSTM(
            config.layers[1],
            return_sequences=False))
        model.add(Dropout(config.dropout))

        model.add(Dense(
            config.n_predictions))
        model.add(Activation("linear"))

        model.compile(loss=config.loss_metric, optimizer=config.optimizer) 

        model.fit(X_train, y_train, batch_size=config.lstm_batch_size, epochs=config.epochs, 
            validation_split=config.validation_split, callbacks=cbs, verbose=True)
        model.save(os.path.join("data", anom['run_id'], "models", anom["chan_id"] + ".h5"))

        return model
```